In [14]:
import atoti as tt
import numpy as np
import pandas as pd
import graphviz

In [15]:
session = tt.create_session()

Deleting existing "Unnamed" session to create the new one.


We convert the stock data into vector. Each value in the vector is a return that correspond to a date in the `returns` df.  
We load the stock with the vector of returns into atoti cube.

In [17]:
returns_table = session.read_csv(
    "returns.csv", keys=["stock"], store_name="Returns", array_sep=";"
)

returns_table.head()

TypeError: __init__() got an unexpected keyword argument 'store_name'

In [ ]:
cube = session.create_cube(returns_table, "Return Cube")

In [ ]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [ ]:
cube.schema

In [ ]:
m

The below formatter will allow us to visualize the first 5 values in each array.

In [ ]:
m["returns_vector.MEAN"].formatter = "ARRAY['|';0:5]"
m["returns_vector.SUM"].formatter = "ARRAY['|';0:5]"

In [ ]:
session.visualize()

### Creating parameter hierarchy for dates
We extracted the dates from the `returns` df and create a parameter hierarchy with it.

In [ ]:
calendar = pd.read_csv("dates.csv")["Date"]
calendar

The parameter hierarchy allows us to have a `Date Index` measure that will be used to retrieve the correspond returns for the day.

In [ ]:
cube.create_static_parameter_hierarchy(
    "Dates", list(calendar), index_measure="Date Index", store_name="Dates"
)
# Setting the hierarchy to non-slicing, as required by
# the max_member aggregation function - see later - this behavior might change in future versions.
h["Dates"].slicing = False

In [ ]:
cube.schema

### Create random weight table

In [ ]:
weight_table = session.create_store(
    types={"stock": tt.type.STRING, "weight": tt.type.NULLABLE_DOUBLE},
    store_name="Weight",
    keys=["stock"],
)

In [ ]:
returns_table.join(weight_table)
cube.schema

### Creating the measure for weight

In [ ]:
m["weight"] = tt.agg.sum(weight_table["weight"])
m["weight"].formatter = "DOUBLE[#,###.00000]"

### Creating function to generate random weight for a given portfolio

In [ ]:
# it generates a random weight across the stocks, that sums up to 1
# creates a scenario for the given weights

def load_weights(scenario, selected_stocks):

    k = np.random.rand(len(selected_stocks))

    selected_stocks["weight"] = k / sum(k)
    selected_stocks.reset_index(inplace=True)

    # load data into atoti cube as a scenario
    weight_table.scenarios[scenario].load_pandas(selected_stocks[["stock", "weight"]])

### Creating the initial weight
Each call to the function will create a new scenario

In [ ]:
# iterate the portfolio of n_stocks by n_weights (gives n_weights scenarios of different weight)
def generate_portfolios(n_stocks, n_weights):
    df_stocks = cube.query(m["contributors.COUNT"], levels=[l["stock"]])
    selected_stocks = df_stocks.sample(n_stocks).copy()

    for i in range(n_weights):
        load_weights(f"weight {i}", selected_stocks)

In [ ]:
generate_portfolios(10, 3)

In [ ]:
session.visualize()

### Create sliding windows

Create a sliding window of size 252. This could be modified using a measure simulation if needed.

In [ ]:
m["Lookback Window"] = 252

In [ ]:
m["Returns sub-vector"] = m["returns_vector.SUM"][
    m["Date Index"] - m["Lookback Window"] + 1 : m["Date Index"] + 1
]

### Computing the summ of the array

In [ ]:
m["p"] = tt.array.mean(m["returns_vector.SUM"])
m["p"].formatter = "DOUBLE[#,###.0000000000]"

The weighted average of the returns

In [ ]:
m["p sliding"] = tt.agg.sum(
    tt.array.sum(m["weight"] * m["Returns sub-vector"]),
    scope=tt.scope.origin(l["stock"]),
)
m["p sliding"].formatter = "DOUBLE[#,###.0000000000]"

In [ ]:
session.visualize()

### Computing mu

In [ ]:
m["mu"] = m["weight"] * m["p"]
m["mu"].formatter = "DOUBLE[#,###.00000]"

In [ ]:
m["mu sliding"] = tt.agg.sum(
    m["weight"] * m["p sliding"], scope=tt.scope.origin(l["stock"])
)
m["mu sliding"].formatter = "DOUBLE[#,###.00000]"

### Computing Standard Deviation

We only compute the std for the stocks that has weights assigned.

In [ ]:
m["Portfolio subvectors"] = tt.agg.sum(
    tt.where((m["weight"] > 0) & (m["weight"] < 1), m["Returns sub-vector"]),
    scope=tt.scope.origin(l["stock"]),
)

In [ ]:
m["Std"] = tt.array.std(m["returns_vector.SUM"])
m["Std"].formatter = "DOUBLE[#,###.00000]"

In [ ]:
m["std sliding"] = tt.array.std(m["Portfolio subvectors"])
m["std sliding"].formatter = "DOUBLE[#,###.00000]"

### Computing Sharpe Ratio

In [ ]:
m["Sharpe Ratio"] = m["mu sliding"] / m["std sliding"]
m["Sharpe Ratio"].formatter = "DOUBLE[#,###.00000]"

In [ ]:
session.visualize()

Compute the maximum Sharpe Ratio for each Weight scenario

In [ ]:
m["Max Sharpe Ratio"] = tt.where(
    m["Sharpe Ratio"] != None,
    tt.total(
        tt.agg.max(m["Sharpe Ratio"], scope=tt.scope.origin(l["Dates"])), h["Dates"]
    ),
)

m["Max Sharpe Ratio"].formatter = "DOUBLE[#,###.00000]"

In [ ]:
session.visualize()

In [ ]:
m["Date max Sharpe"] = tt.agg.max_member(m["Sharpe Ratio"], l["Dates"])

Based on the max Sharpe Ratio, select the needed weight

In [ ]:
session.visualize("Weight 0 simulation")